# Clustering Crypto

In [26]:
# Initial imports

import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

### Fetching Cryptocurrency Data

In [2]:
# Use the following endpoint to fetch json data
#url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [3]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [4]:
# Alternatively, use the provided csv file:
# Create a DataFrame
file_path = Path("Resources/crypto_data.csv")
crypto_data = pd.read_csv(file_path)
crypto_data.head(10)


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


### Data Preprocessing

In [5]:
crypto_data.dtypes

Unnamed: 0          object
CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [6]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_data=crypto_data.drop(columns=["Unnamed: 0"])
crypto_data.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [7]:
# Keep only cryptocurrencies that are trading
crypto_data=crypto_data.loc[crypto_data['IsTrading'] == True]
crypto_data.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [8]:
# Keep only cryptocurrencies with a working algorithm
crypto_data = crypto_data.loc[crypto_data['Algorithm'] != 'N/A']

crypto_data.head(15)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [9]:
# Remove the "IsTrading" column
crypto_data=crypto_data.drop(columns=["IsTrading"])
crypto_data.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [10]:
# Remove rows with at least 1 null value
crypto_data.dropna(axis='columns')
crypto_data.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [11]:
# Remove rows with cryptocurrencies having no coins mined
crypto_data = crypto_data.loc[crypto_data['TotalCoinsMined'] != 0]
crypto_data.head(15)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000
10,Dash,X11,PoW/PoS,9.031294e+06,22000000


In [12]:
# Drop rows where there are 'N/A' text values
crypto_data = crypto_data[crypto_data != 'N/A'].dropna()

In [13]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
crypto_data_coinname =crypto_data["CoinName"].to_frame()
crypto_data_coinname

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum
...,...
1238,ZEPHYR
1242,Gapcoin
1245,Beldex
1246,Horizen


In [14]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_data=crypto_data.drop(columns=["CoinName"])
crypto_data.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
9,Scrypt,PoW,6.303924e+07,84000000
10,X11,PoW/PoS,9.031294e+06,22000000
11,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethash,PoW,1.133597e+08,210000000
13,Equihash,PoW,7.383056e+06,21000000


In [15]:
# Create dummy variables for text features
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
crypto_data['Algorithm'] = le.fit_transform(crypto_data['Algorithm'])
crypto_data['ProofType'] = le.fit_transform(crypto_data['ProofType'])
crypto_data.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,53,16,4.199995e+01,42
2,53,16,1.055185e+09,532000000
5,67,16,2.927942e+10,314159265359
7,47,13,1.792718e+07,21000000
8,20,13,1.076842e+08,0


In [16]:
# Standardize data
X=pd.DataFrame(crypto_data)
X_scaled = StandardScaler().fit_transform(X)



### Reducing Dimensions Using PCA

In [17]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [18]:
# Create a DataFrame with the principal components data
pcs_df= pd.DataFrame(
    data=crypto_pca, columns=["PC1", "PC2", "PC3"]
)
pcs_df.head()

,PC1,PC2,PC3
0,-0.412665,0.823150,0.357227
1,-0.391306,0.827873,0.358336
2,3.143152,2.186748,0.487273
3,-0.190301,-0.003829,-0.043688
4,-0.051013,-1.161387,1.020473


In [19]:
# explained variance
pca.explained_variance_ratio_
print(pca.explained_variance_ratio_.sum())

0.927070890057295


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [20]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\Users\sprin\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


:Curve   [k]   (inertia)

Running K-Means with k=4

In [21]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
pcs_df['class']= model.labels_
# Create return DataFrame with predicted clusters
clustered_df = pd.concat([crypto_data,crypto_data_coinname,pcs_df], axis=1)
clustered_df.dropna(inplace=True)
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC1,PC2,PC3,class
0,53.0,16.0,4.199995e+01,42,42 Coin,-0.412665,0.823150,0.357227,1.0
2,53.0,16.0,1.055185e+09,532000000,404Coin,3.143152,2.186748,0.487273,1.0
5,67.0,16.0,2.927942e+10,314159265359,EliteCoin,-0.219679,0.253752,-0.280044,1.0
7,47.0,13.0,1.792718e+07,21000000,Bitcoin,-0.011450,-1.504754,1.335799,0.0
8,20.0,13.0,1.076842e+08,0,Ethereum,-0.048798,-1.160925,1.020929,0.0
...,...,...,...,...,...,...,...,...,...
516,67.0,8.0,2.500124e+06,2500124,RoyalCoin,0.509486,-2.078313,-2.779467,2.0
522,68.0,16.0,1.000000e+08,100000000,GanjaCoin V2,0.696039,-3.504453,-1.080402,2.0
523,41.0,8.0,1.781868e+07,301000000,TeamUP,0.153119,-0.948063,-1.104022,2.0
525,49.0,16.0,1.082163e+09,7506000000,LanaCoin,0.158777,-1.210829,-0.869641,2.0


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [22]:
# Scale data to create the scatter plot
scaler = MinMaxScaler()

In [23]:
# Scale data to create the scatter plot
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
scaler = MinMaxScaler()

normalize_columns = ['TotalCoinsMined', 'TotalCoinSupply']
x = crypto_data[normalize_columns].values
x_scaled = scaler.fit_transform(x)

df_tradable = pd.DataFrame(x_scaled, columns=normalize_columns, index=crypto_data.index).reset_index()
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
df_tradable.hvplot.scatter(
    x='TotalCoinsMined',
    y='TotalCoinSupply',
    hover_cols=['index'],
)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,index)

#### Table of Tradable Cryptocurrencies

In [24]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType',
                                  'TotalCoinSupply', 'TotalCoinsMined',
                                  'class']
                         )

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [25]:
# Print the total number of tradable cryptocurrencies
print(f'The total number of tradable cryptocurrenices is: {len(clustered_df)}')

The total number of tradable cryptocurrenices is: 205
